In [76]:
import os
import re
from math import log, sqrt

In [67]:
def nice_format(value):
    return "{:,}".format(value)

In [13]:
base_path = "./pa1-data/"
files_name = dict()

for i in range(10):
    dir_path = base_path + str(i)
    files = os.listdir(dir_path)
    files = [file for file in files if file != '.DS_Store']
    files_name[i] = files

In [16]:
l = os.listdir(base_path)
[file for file in l if file != '.DS_Store']

['9', '0', '7', '6', '1', '8', '4', '3', '2', '5']

In [17]:
common_words = []
with open("CACM/common_words", "r") as cw:
    common_words = [ l.replace("\n", "") for l in cw.readlines()]

In [37]:
def get_tokens_document(base_path, dir_path, file_name):
    with open(base_path + dir_path + file_name, "r") as f:
        current_document_tokens = []

        lines = f.readlines()
        for line in lines:
            line.replace("\n", "")
            lower_line = line.lower()
            list_split = re.split("\W+", lower_line)
            for t in list_split:
                if t not in common_words and t != "":
                    current_document_tokens += [t]

        return current_document_tokens

In [48]:
base_path = "./pa1-data/"

docid_names = {}
tokens = {}

doc_id = 1
for directory in files_name.keys():
    dir_path = str(directory) + '/'
    for file_name in files_name[directory]:
        docid_names[doc_id] = dir_path + file_name
        tokens[doc_id] = get_tokens_document(base_path, dir_path, file_name)
        doc_id += 1

In [68]:
nb_documents = doc_id
print("Number of document: " + nice_format(nb_documents))

Number of document: 98,999


In [50]:
vocabulaire = set()

for key in tokens.keys():
    vocabulaire.update(tokens[key])

In [65]:
taille_vocabulaire = len(vocabulaire)
print("Taille vocabulaire: " + nice_format(taille_vocabulaire))

Taille vocabulaire: 325,984


In [66]:
nb_tokens = sum([len(tokens[x]) for x in tokens.keys()])
print("Nombre de tokens: " + nice_format(nb_tokens))

Nombre de tokens: 17,719,205


### Question 3

In [70]:
tokens_moitie = {}
for i in range(1, int(nb_documents/2) + 1):
    tokens_moitie[i] = tokens[i]

In [72]:
nb_tokens_moitie = sum([len(tokens_moitie[x]) for x in tokens_moitie])
print("Nombre de tokens pour la moitie des documents: " + nice_format(nb_tokens_moitie))

Nombre de tokens pour la moitie des documents: 9,841,885


In [73]:
vocabulaire_moitie = set()

for key in tokens_moitie.keys():
    vocabulaire_moitie.update(tokens_moitie[key])

In [74]:
taille_vocabulaire_moitie = len(vocabulaire_moitie)
print("Nombre de tokens pour la moitie des documents: " + nice_format(taille_vocabulaire_moitie))

Nombre de tokens pour la moitie des documents: 184,925


In [77]:
b = log(float(taille_vocabulaire)/taille_vocabulaire_moitie)/log(float(nb_tokens)/nb_tokens_moitie)
print ("b=" + str(b))

b=0.9641091946757916


In [78]:
k = taille_vocabulaire/pow(nb_tokens,b)
print ("k=" +str(k))

k=0.03348918300041858


### Question 4

In [80]:
print (nice_format(k * pow(1e6,b)))

20,396.69119331342


### Question 5

In [81]:
frequences = {}

for key in tokens.keys():
    for token in tokens[key]:
        if token not in frequences.keys():
            frequences[token] = 1
        else:
            frequences[token] += 1

In [82]:
sorted_by_value = sorted(frequences.items(), key=lambda kv: kv[1])

In [83]:
sorted_by_value.reverse()

In [84]:
sorted_by_value[:10]

[('stanford', 363488),
 ('university', 134427),
 ('1', 128831),
 ('research', 105426),
 ('home', 86017),
 ('center', 83196),
 ('2', 80492),
 ('page', 80104),
 ('contact', 78372),
 ('search', 70452)]

In [85]:
xs = [i+1 for i in range(len(sorted_by_value))]
ys = [y[1] for y in sorted_by_value]

In [86]:
ln_xs = [log(i+1) for i in range(len(sorted_by_value))]
ln_ys = [log(y[1]) for y in sorted_by_value]

In [89]:
# import matplotlib.pyplot as plt

In [90]:
# plt.plot(ln_xs, ln_ys, 'bo', label='Frequence')
# plt.legend()
# plt.title('ln(frequence)=f(ln(rang))')

## Question 2.2

In [91]:
term_ids = {}
for (i,k) in enumerate(frequences):
    term_ids[k] = i

term_ids_inverse = {v: k for k, v in term_ids.items()}

In [92]:
def frequence_token(token, document_id):
    return tokens[document_id].count(token) / len(tokens[document_id])

In [93]:
taille_bloc = 100
splitted_tokens = {}
for (i,k) in enumerate(tokens):
    if i//taille_bloc in splitted_tokens:
        splitted_tokens[i//taille_bloc][k] = tokens[k]
    else:
        splitted_tokens[i//taille_bloc] = {}
        splitted_tokens[i//taille_bloc][k] = tokens[k]

In [94]:
index_inverse = {}
for bloc in splitted_tokens.values():
    buffer = []
    for (i,k) in enumerate(bloc):
        for pair in bloc[k]:
            buffer += [(term_ids[pair], k)]
    sorted_index = sorted(buffer, key=lambda kv: kv[0])
    for pair in sorted_index:
        term_id, doc_id = pair
        if term_id in index_inverse:
            index_inverse[term_id] += [(doc_id, frequence_token(term_ids_inverse[term_id], doc_id))]
        else:
            index_inverse[term_id] = [(doc_id, frequence_token(term_ids_inverse[term_id], doc_id))]

In [95]:
def boolean_request_and(token1, token2):
    id_1 = term_ids[token1]
    id_2 = term_ids[token2]
        
    list1 = [pair[0] for pair in index_inverse[id_1]] if id_1 in index_inverse else []
    list2 = [pair[0] for pair in index_inverse[id_2]] if id_2 in index_inverse else []
        
    return set(list1).intersection(set(list2))

In [96]:
def boolean_request_or(token1, token2):
    id_1 = term_ids[token1]
    id_2 = term_ids[token2]
    
    list1 = [pair[0] for pair in index_inverse[id_1]] if id_1 in index_inverse else []
    list2 = [pair[0] for pair in index_inverse[id_2]] if id_2 in index_inverse else []
        
    return set(list1).union(set(list2)) 

In [97]:
document_ids = set(tokens.keys())
def boolean_request_not(token1):
    id_1 = term_ids[token1]
    
    list1 = [pair[0] for pair in index_inverse[id_1]] if id_1 in index_inverse else []
        
    return document_ids.difference(set(list1)) 

In [98]:
def boolean_request_not_ensemble(set1):
    return document_ids.difference(set1) 

In [99]:
def requete_arbre(arbre):
    if len(arbre) == 1:
        id_1 = term_ids[arbre[0]]
        list1 = [pair[0] for pair in index_inverse[id_1]] if id_1 in index_inverse else []
        
        return set(list1)
    
    operator = arbre[1]
    if operator == 'AND':
        return requete_arbre(arbre[0]).intersection(requete_arbre(arbre[2]))
    elif operator == 'OR':
        return requete_arbre(arbre[0]).union(requete_arbre(arbre[2]))
    elif operator == 'NOT':
        return boolean_request_not_ensemble(requete_arbre(arbre[0]))

In [100]:
taille_documents = {}
for (i,k) in enumerate(tokens):
    taille_documents[k] = sqrt(sum([pow(frequence_token(tok, k),2) for tok in tokens[k]]))

In [102]:
idfs = {}
for k in index_inverse:
    idfs[k] = log(nb_documents/len(index_inverse[k]),10)

In [103]:
idfs[1]

3.3424182939871017

In [104]:
def cosinus_tf(requete):
    similarites = {}
    norme_requete = sqrt(sum([pow(requete.count(terme)/len(requete),2) for terme in requete]))
    for terme in requete:
        terme_id = term_ids[terme]
        freq_terme_in_requete = requete.count(terme)/len(requete)
        for (documentId, freq) in index_inverse[terme_id]:
            if documentId in similarites:
                similarites[documentId] += freq*freq_terme_in_requete/(norme_requete*taille_documents[documentId])
            else:
                similarites[documentId] = freq*freq_terme_in_requete/(norme_requete*taille_documents[documentId])
    
    return sorted(similarites.items(), key=lambda kv: kv[1], reverse=True)

In [105]:
cosinus_tf(['assignment', 'restrictions'])[:10]

[(11769, 3.2934186664251794),
 (604, 3.2392726799655573),
 (2611, 3.154316277589963),
 (53212, 2.374536067453556),
 (52718, 2.0561587888491846),
 (27159, 1.9521208050960102),
 (72311, 1.916054224539098),
 (71569, 1.7023689888471423),
 (17494, 1.5141278278751513),
 (16384, 1.4288732936180237)]

In [106]:
def cosinus_tf_idf_log(requete):
    similarites = {}
    norme_requete = sqrt(sum([pow(requete.count(terme)/len(requete),2) for terme in requete]))
    for terme in requete:
        terme_id = term_ids[terme]
        poids_terme_in_requete = requete.count(terme)/(len(requete)*idfs[terme_id])
        for (documentId, freq) in index_inverse[terme_id]:
            poids = 1 + log(freq/idfs[terme_id])
            if documentId in similarites:
                similarites[documentId] += poids*poids_terme_in_requete/(norme_requete*taille_documents[documentId])
            else:
                similarites[documentId] = poids*poids_terme_in_requete/(norme_requete*taille_documents[documentId])
    return sorted(similarites.items(), key=lambda kv: kv[1], reverse=True)

In [107]:
cosinus_tf_idf_log(['assignment', 'restrictions'])[:10]

[(37457, -1.0728672793497671),
 (68227, -1.2400616681687582),
 (86719, -1.3033332676845009),
 (75882, -1.3148055803698344),
 (92732, -1.4338585574040932),
 (33682, -1.5946774102422079),
 (65588, -1.6563233290648487),
 (83735, -1.8066779930830594),
 (68835, -1.8099225548681708),
 (44860, -1.8295746057360716)]